# Short cadence observations of HAT-P-7 b

HAT-P-7 b ([Pál et al. 2008](https://ui.adsabs.harvard.edu/abs/2008ApJ...680.1450P/abstract)) is a hot Jupiter orbiting a bright F6V host in the Kepler field. Let's take a look at short cadence (one-minute) Kepler observations from the first three Kepler quarters to see the secondary eclipse of the planet, when the planet's flux disappears as the planet is eclipsed by the host star.

First we download the observations with `lightkurve`:

In [ ]:
import astropy.units as u
from lightkurve import search_lightcurve
from astropy.time import Time
from lcviz import LCviz

# Download three quarters of short cadence observations from Kepler:
lc = search_lightcurve(
    target="HAT-P-7", 
    mission="Kepler", 
    quarter=[1, 2, 3], 
    cadence="short", 
    author="Kepler"
).download_all().stitch(
    # stitch together the quarters after normalizing each one:
    lambda x: x.normalize()
)

Now load the light curve's 300,000 flux measurements:

In [ ]:
# Load the light curve into LCviz:
lcviz = LCviz()
lcviz.load_data(lc, data_label=f"HAT-P-7")
lcviz.show()

In [ ]:
# use the flatten plugin to flatten each quarter:
flatten = lcviz.plugins['Flatten']

for dataset in flatten.dataset.choices:
    flatten.dataset = dataset
    flatten.window_length = 2000
    flatten.flatten();

Now let's phase-fold the observations with the ephemeris of the planet using the `Ephemeris` plugin:

In [ ]:
# planet parameters from Morris et al. 2013:
# https://ui.adsabs.harvard.edu/abs/2013ApJ...764L..22M/abstract

# get the origin of the time axis in LCviz:
time_coordinates = lcviz.app.data_collection[0].coords
reference_time = time_coordinates.reference_time

period = 2.204737  # days
epoch = Time(2454954.357463, format='jd')

# phase-fold the transit light curve in an ephemeris viewer:
eph = lcviz.plugins['Ephemeris']
eph.period = period
eph.t0 = (
    (epoch - reference_time).to_value(time_coordinates.unit) % eph.period
)

We can easily see the secondary eclipse if we bin the phased observations with the `Binning` plugin:

In [ ]:
binning = lcviz.plugins['Binning']
binning.n_bins = 150
binning.keep_active = False
binning.ephemeris = 'default'
binning.bin();

and adjust the axis limits and colors:

In [ ]:
viewer = lcviz.app.get_viewer('flux-vs-phase:default')
viewer.state.y_min = 0.99993
viewer.state.y_max = 1.00005

plot_options = lcviz.plugins['Plot Options']
plot_options.viewer = 'flux-vs-phase:default'
plot_options.layer = 'HAT-P-7[pdcsap_flux]'
plot_options.marker_opacity = 0.1
plot_options.marker_size = 1

plot_options.layer = 'binned HAT-P-7[pdcsap_flux]:default'
plot_options.marker_size = 50
plot_options.marker_color = 'r'